In [ ]:
# !pip install IPython
from IPython.display import display, HTML, Image

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day16.git #코랩 사용

# [모델 평가 지표]
## 회귀 모델 평가 지표

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 데이터 전처리 패키지 '''
import numpy as np
import pandas as pd

''' 기계학습 모델 구축 및 평가 패키지 '''
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

''' 데이터 시각화 패키지 '''
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

''' 경고 숨기기 '''
import warnings
warnings.filterwarnings(action='ignore')

''' 한글 폰트 설정 '''
plt.rc('font', family='Malgun Gothic')

## 2. 분석 데이터셋: Toyota Corolla 

<center><a href='https://www.kaggle.com/klkwak/toyotacorollacsv'> https://www.kaggle.com/klkwak/toyotacorollacsv </a></center>


### 설명변수 (X)
- Mfg_Month
- Mfg_Year
- KM
- Fuel_Type
...
- Radio_cassette
- Tow_Bar

### 반응변수 (Y)
- Price

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/LG_time_series_day16/data/ToyotaCorolla.csv')
# data = pd.read_csv('./data/ToyotaCorolla.csv') # 로컬

In [ ]:
# 데이터 크기 및 사용 변수
print("데이터 크기 : ", data.shape)

In [ ]:
### 불필요한 변수 제거

data = data.drop(labels=['Id', 'Model'], axis=1)
print("불필요한 변수 제거 후 데이터 크기 : ", data.shape)

In [ ]:
# 데이터 예시
print("X samples : ", data[:3])

## 3. 데이터 전처리

$\mathbf{X}$: 설명변수/입력변수 <br>
$\mathbf{y}$: 반응변수/출력변수

- X, Y 데이터 프레임 만들기

In [ ]:
X = data.drop(labels='Price', axis=1)
y = data['Price']

In [ ]:
display(X.head())
display(y.head())

### 범주형 설명변수에 대한 Dummy 변수 생성

In [ ]:
data['Fuel_Type']

In [ ]:
data.groupby('Fuel_Type')['Fuel_Type'].count()

In [ ]:
X = X.drop(labels='Fuel_Type', axis=1)

In [ ]:
X_dummy = pd.get_dummies(data=data['Fuel_Type'], prefix='Fuel_Type', drop_first=True)

In [ ]:
X_dummy

In [ ]:
X_dummied = pd.concat(objs=[X,X_dummy], axis=1)

In [ ]:
X_dummied.head()

### X, Y 데이터 프레임 합치기

In [ ]:
data = pd.concat([X_dummied, y], axis=1)
data.head()

## 4. 탐색적 데이터 분석

- sns.pairplot을 이용하여 설명변수의 분포(히스토그램) 및 설명변수들 간의 관계(산점도)를 그리고, 특성을 파악

In [ ]:
# 변수가 매우 많아 여기서는 실행 X
# sns.pairplot(data)
# plt.show()

- 일부 변수에 대해 그래프 확인

In [ ]:
# "Age_08_04", "Mfg_Year", "Fuel_Type_Petrol", "Price" 변수 확인
plt.figure(figsize=(10, 10))
sns.pairplot(data[['Age_08_04', 'Mfg_Year', 'Fuel_Type_Petrol', 'Price']])
plt.show()

### 설명변수 간 상관계수(correlation coefficient) 확인

- data.corr(): 데이터 상관관계
- sns.heatmap(데이터, cmap= colormap 이름, annot=히트맵 안에 표시할 데이터)


In [ ]:
# 상관계수 행렬
data.corr()

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(data.corr(), cmap=sns.color_palette("coolwarm", 10), annot=True)
# sns.heatmap(data.corr(), cmap="jet", annot=True) ## Color map 변경 
# sns.heatmap(data.corr(), cmap=sns.color_palette("coolwarm", 10), annot=False) ## Annot 변경
plt.show()

- 특정 값 이상 상관성을 갖는 조합을 확인

In [ ]:
threshold_corr = 0.5

corrmat = data.corr()
select_features = corrmat.index[abs(corrmat["Price"]) >= threshold_corr]

plt.figure(figsize=(10,10))
sns.heatmap(data[select_features].corr(), cmap=sns.color_palette("coolwarm", 10), annot=True)

## 5. 모델링

### statsmodels의 OLS를 사용하여 선형회귀분석 시행 (OLS: Ordinary Least Squares)
 - OLS: 가장 기본적인 결정론적 선형 회귀 방법으로 잔차제곱합(RSS: Residual Sum of Squares)을 최소화하는 가중치(β1, β2 ...)를 구하는 방법
 - 모델 선언: model = sm.OLS(Y데이터, X데이터)   
 - 모델 학습: model_trained = model.fit()

In [ ]:
Image('/content/LG_time_series_day16/image/intro1.png')

In [ ]:
# 상수항(b0) 정의
data = sm.add_constant(data)

# 입력변수(X)와 출력변수(Y) 정의
model = sm.OLS(data['Price'], data.drop(['Price'], axis=1))

In [ ]:
# 최적의 파라미터 탐색
model_trained = model.fit()

### QQ plot을 통해 선형 모형의 정규성 가정을 시각적으로 확인

In [ ]:
# 확률오차의 정규성 확인

model_residuals = model_trained.resid

plt.rcParams['axes.unicode_minus'] = False # 음수 폰트 깨짐 방지
fig, ax = plt.subplots(1,1)
fig.set_figheight(8) 
fig.set_figwidth(10) 

sm.ProbPlot(model_residuals).qqplot(line='s', color='#1f77b4', ax=ax)
ax.title.set_text('Q-Q Plot')

In [ ]:
# 확률오차의 등분산성 확인

model_fitted_y= model_trained.fittedvalues

fig, ax = plt.subplots(1, 1)
fig.set_figheight(8)
fig.set_figwidth(10)

# lowess : 평활화 (곡면에 피팅)

sns.residplot(data=data, x= model_fitted_y, y= 'Price', lowess=True, scatter_kws={'alpha': 0.5}, 
              line_kws={'color':'red'}, ax=ax)
ax.title.set_text('Residuals vs Fitted')
ax.set(xlabel='Fitted values', ylabel='Residuals')

## 6. 모델 해석
- R-squared (결정계수,coefficient of determination):모형의 성능
- coef (회귀계수): X가 한단위 증가할 때 Y의 변화량
- P>[t] (p-value):0.05(유의수준) 이하일 때 변수가 유의미

# __모델 해석__

In [ ]:
display(model_trained.summary())

In [ ]:
Image('content/LG_time_series_day16/image/intro2.PNG')

## 7. 회귀모델 성능 평가

In [ ]:
# 실제값
true = y

# 예측값
pred = model_trained.fittedvalues

In [ ]:
fig_values = np.concatenate([true.values.squeeze(), pred.values.squeeze()])
vmin = np.min(fig_values) * 0.95
vmax = np.max(fig_values) * 1.05

plt.figure(figsize=(8, 8))
plt.title('실제값 vs. 모델 출력 값', size=18)
plt.scatter(y, pred)
plt.plot([vmin, vmax], [vmin, vmax], color='grey', linestyle='dashed')
plt.xlabel('실제값', size=16)
plt.ylabel('모델 출력 값', size=16)
plt.show()

### 다양한 회귀 모델 예측 성능 평가 지표들

- 실제값 : $y_{i}$
- 예측값 : $\hat{y}_{i}$

#### Mean Squared Error (평균 제곱 오차): $\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}$

####  Root Mean Squared Error (제곱근 평균 제곱 오차): $\sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}}$

####  Mean Absolute Error (평균 절대 오차): $\frac{1}{n} \sum_{i=1}^{n} |\hat{y}_{i}- y_{i}|$

####  Mean Absolute Percentage Error (평균 절대 백분율 오차): $\frac{100}{n} \sum_{i=1}^{n} |\frac{\hat{y}_{i}- y_{i}}{\hat{y}_{i}}|$

####  R squared (결정계수 $({r})^{2}$): $\frac{SSR}{SST} = 1- \frac{SSR}{SST}%$

In [ ]:
# MAPE def.
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [ ]:
MSE_value = mean_squared_error(true, pred)
RMSE_value = np.sqrt(mean_squared_error(true, pred))
MAE_value = mean_absolute_error(true, pred)
MAPE_value = mean_absolute_percentage_error(true, pred)
r2_value = r2_score(true, pred)

print("Training MSE : {:.3f}".format(MSE_value))
print("Training RMSE : {:.3f}".format(RMSE_value))
print("Training MAE : {:.3f}".format(MAE_value))
print("Training MAPE : {:.3f}".format(MAPE_value))
print("Training R2 : {:.3f}".format(r2_value))

---